# 07. Regression (Supervised) for PM2.5 (Tabular Time-Lag Features)
Mục tiêu:
- Biến bài toán chuỗi thời gian thành bài toán **hồi quy có giám sát**: dự đoán PM2.5(t+h) từ đặc trưng tại thời điểm t.
- Thấy rõ: **leakage** và vì sao phải split theo thời gian.
- So sánh tư duy hồi quy (feature-based) vs ARIMA (time-series-based).


In [ ]:
from pathlib import Path
import sys

# ===== PARAMETERS =====
USE_UCIMLREPO = False

# Path to the raw ZIP (relative to project root)
RAW_ZIP_PATH_REL = 'data/raw/PRSA2017_Data_20130301-20170228.zip'

# Tự động tìm PROJECT_ROOT (thư mục chứa src/)
cwd = Path.cwd().resolve()
PROJECT_ROOT = cwd
while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / 'src').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if not (PROJECT_ROOT / 'src').exists():
    raise FileNotFoundError("Không tìm thấy thư mục 'src' trong cây thư mục hiện tại.")

# Bảo đảm import được package trong src
sys.path.insert(0, str(PROJECT_ROOT))

# Chuẩn hoá đường dẫn tuyệt đối
RAW_ZIP_PATH = (PROJECT_ROOT / RAW_ZIP_PATH_REL).resolve()
RAW_ZIP_ABS = str(RAW_ZIP_PATH)

LAG_HOURS = [1, 3, 24]
HORIZON = 1              # dự đoán trước bao nhiêu giờ
TARGET_COL = 'PM2.5'

OUTPUT_REG_DATASET_PATH = 'data/processed/07_dataset_for_regression.parquet'
CUTOFF = '2017-01-01'

MODEL_OUT = '07_regressor.joblib'
METRICS_OUT = '07_regression_metrics.json'
PRED_SAMPLE_OUT = '07_regression_predictions_sample.csv'

print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"RAW_ZIP_PATH: {RAW_ZIP_PATH}")

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.classification_library import Paths
from src.regression_library import (
    run_prepare_regression_dataset,
    run_train_regression,
)

paths = Paths(project_root=PROJECT_ROOT)

# Tạo thư mục images để lưu biểu đồ
IMAGES_PATH = PROJECT_ROOT / 'images'
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

# Tạo thư mục data/processed để lưu CSV
DATA_PATH = PROJECT_ROOT / 'data' / 'processed'
DATA_PATH.mkdir(parents=True, exist_ok=True)

print('PROJECT_ROOT =', PROJECT_ROOT)
print('RAW_ZIP_ABS =', RAW_ZIP_ABS)

## 1) Tạo dataset hồi quy (lag features + time features + y = PM2.5(t+h))
Trong lab, phần này giúp sinh viên hiểu cách tạo supervised dataset từ time series.

In [ ]:

print(f"file_name: ", OUTPUT_REG_DATASET_PATH.split("/")[-1])

In [ ]:
out_path = run_prepare_regression_dataset(
    paths=paths,
    use_ucimlrepo=USE_UCIMLREPO,
    raw_zip_path=RAW_ZIP_ABS,
    lag_hours=LAG_HOURS,
    horizon=HORIZON,
    target_col=TARGET_COL,
    file_name=str( OUTPUT_REG_DATASET_PATH.split("/")[-1])
)
print('Saved:', out_path)

## 2) Quick EDA cho dataset hồi quy
Gợi ý câu hỏi ra quyết định:
- Tỉ lệ missing ở các feature lag? (thường thiếu ở đầu chuỗi)
- PM2.5 có phân phối lệch (skew) không? -> cân nhắc log/clip (tuỳ chọn)
- Có khác biệt theo *giờ trong ngày* / *ngày trong tuần* không? (seasonality)


In [ ]:
ds_path = (PROJECT_ROOT / OUTPUT_REG_DATASET_PATH).resolve()

print(f"ds_path: f{ds_path}")

df = pd.read_parquet(ds_path)

print(df.shape)
display(df.head())

# Lưu regression dataset sample
df.head(100).to_csv(DATA_PATH / '07_regression_dataset_sample.csv', index=False)
print('Saved:', DATA_PATH / '07_regression_dataset_sample.csv')

missing = df.isna().mean().sort_values(ascending=False).head(15)
display(missing)

# Lưu missing values
missing.to_frame(name='missing_rate').to_csv(DATA_PATH / '07_missing_values.csv')
print('Saved:', DATA_PATH / '07_missing_values.csv')

fig, ax = plt.subplots(figsize=(10, 6))
pd.Series(df[TARGET_COL]).dropna().plot(kind='hist', bins=60, ax=ax, color='#1F62FF')
ax.set_title(f'Distribution of {TARGET_COL} (raw)', fontsize=14, fontfamily='Arial')
ax.set_xlabel(TARGET_COL, fontsize=11, fontfamily='Arial')
ax.set_ylabel('Frequency', fontsize=11, fontfamily='Arial')
ax.tick_params(axis='both', labelsize=11)
plt.tight_layout(pad=1.5)

# Lưu biểu đồ
plt.savefig(IMAGES_PATH / '07_target_distribution.png', dpi=300, bbox_inches='tight')
print('Saved:', IMAGES_PATH / '07_target_distribution.png')
plt.show()

## 3) Train/Test theo thời gian + train regressor
Lưu ý: mô hình hồi quy ở đây là **feature-based** (dùng lag + thời tiết).
Phần dự báo chuỗi thời gian *thuần* sẽ làm bằng ARIMA ở notebook kế tiếp.

In [ ]:
out = run_train_regression(
    paths=paths,
    cutoff=CUTOFF,
    model_out=MODEL_OUT,
    metrics_out=METRICS_OUT,
    preds_out=PRED_SAMPLE_OUT,
    dataset_file=OUTPUT_REG_DATASET_PATH.split("/")[-1],
)

print('Metrics:')
print(json.dumps(out['metrics'], ensure_ascii=False, indent=2))
pred_df = out['pred_df']
display(pred_df.head())

# Lưu regression predictions sample
pred_df.head(100).to_csv(DATA_PATH / '07_regression_predictions.csv', index=False)
print('Saved:', DATA_PATH / '07_regression_predictions.csv')

# Plot a small window for storytelling
sample = pred_df.dropna().iloc[:500].copy()
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(sample['datetime'], sample['y_true'], label='Actual', color='#1F62FF')
ax.plot(sample['datetime'], sample['y_pred'], label='Predicted', color='#FF351F')
ax.set_title('Regression: Actual vs Predicted (sample)', fontsize=14, fontfamily='Arial')
ax.set_xlabel('Datetime', fontsize=11, fontfamily='Arial')
ax.set_ylabel('PM2.5', fontsize=11, fontfamily='Arial')
ax.tick_params(axis='both', labelsize=11)
ax.legend(fontsize=11)
plt.tight_layout(pad=1.5)

# Lưu biểu đồ
plt.savefig(IMAGES_PATH / '07_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
print('Saved:', IMAGES_PATH / '07_actual_vs_predicted.png')
plt.show()